In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
DATA_PATH = "/content/drive/MyDrive/final_pj/final_project_data/"
DATA_PATH

'/content/drive/MyDrive/final_pj/final_project_data/'

In [100]:
SEED = 42

In [138]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [139]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.2_1107.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.2_1107.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 1455), (12225, 1455))

In [103]:
train_ft.isnull().sum().sum()

0

In [104]:
test_ft.isnull().sum().sum()

0

In [140]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1454), (12225, 1454))

In [141]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,238


In [107]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [128]:
%pip install category_encoders

In [109]:
# all_countencoder
#import category_encoders as ce
#
#enc = ce.CountEncoder(cols=cols)
#tmp = enc.fit_transform(train_ft[cols])
#
#for col in cols:
#    train_ft[f'{col}_enc'] = tmp[col]
#
#tmp = enc.transform(test_ft[cols])
#for col in cols:
#    test_ft[f'{col}_enc'] = tmp[col]
#
#train_ft.shape, test_ft.shape

In [110]:
# all_onehotencoder
#import category_encoders as ce
#enc = ce.one_hot.OneHotEncoder()
#tmp = enc.fit_transform(train_ft[cols])
#train_ft = pd.concat([train_ft,tmp],axis =1)
#
#tmp = enc.transform(test_ft[cols])
#test_ft = pd.concat([test_ft,tmp],axis =1)
#
#train_ft.shape, test_ft.shape

In [142]:
# 주구매지점-onehot / 주구매_대중분류 - countencoder
import category_encoders as ce
enc1 = ce.one_hot.OneHotEncoder()
tmp = enc1.fit_transform(train_ft["주구매지점"])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc1.transform(test_ft["주구매지점"])
test_ft = pd.concat([test_ft,tmp],axis =1)


enc2 = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc2.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc2.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc2.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc2.transform(test_ft[["주구매_대분류"]])
train_ft.shape, test_ft.shape

((14940, 1460), (12225, 1460))

In [112]:
# 주구매지점-labelencoer / 주구매_대중분류 - countencoder
#from sklearn.preprocessing import LabelEncoder
#import category_encoders as ce
#enc1 = LabelEncoder()
#train_ft["주구매지점"] = enc1.fit_transform(train_ft["주구매지점"])
#test_ft["주구매지점"] = enc1.transform(test_ft["주구매지점"])
#
#enc2 = ce.count.CountEncoder()
#train_ft["주구매_중분류_cnt"] = enc2.fit_transform(train_ft[["주구매_중분류"]])
#test_ft["주구매_중분류_cnt"] = enc2.transform(test_ft[["주구매_중분류"]])
#
#train_ft["주구매_대분류_cnt"] = enc2.fit_transform(train_ft[["주구매_대분류"]])
#test_ft["주구매_대분류_cnt"] = enc2.transform(test_ft[["주구매_대분류"]])
#train_ft.shape, test_ft.shape

In [113]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [114]:
#train_ft = train_ft.drop(columns=['주구매_대분류', '주구매_중분류'])
#test_ft = test_ft.drop(columns=['주구매_대분류', '주구매_중분류'])
#train_ft.shape, test_ft.shape

In [143]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1457), (12225, 1457))

In [144]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [145]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

In [118]:
# all_minmaxscaler
#scaler = MinMaxScaler()
#scaler.fit(train_ft)
#train_ft[train_ft.columns] = scaler.transform(train_ft)
#test_ft[test_ft.columns] = scaler.transform(test_ft)

In [146]:
# all_standardscaler
scaler = StandardScaler()
scaler.fit(train_ft)
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [120]:
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,주구매_년,주구매_월,주구매_시간대,일별평균구매횟수,거래개월수,9시_12시_구매비율,12시_15시_구매비율,15시_18시_구매비율,18시_21시_구매비율,9시_12시_구매횟수,12시_15시_구매횟수,15시_18시_구매횟수,18시_21시_구매횟수,월초_구매비율,월말_구매비율,월초_구매횟수,월말_구매횟수,웨딩성수기_구매비율,웨딩성수기_구매횟수,1월_구매비율,2월_구매비율,3월_구매비율,4월_구매비율,5월_구매비율,6월_구매비율,7월_구매비율,8월_구매비율,9월_구매비율,10월_구매비율,11월_구매비율,12월_구매비율,1월_구매횟수,2월_구매횟수,3월_구매횟수,4월_구매횟수,5월_구매횟수,6월_구매횟수,7월_구매횟수,8월_구매횟수,...,공휴일_대분류_가정용품_총구매금액,공휴일_대분류_골프_유니캐쥬얼_총구매금액,공휴일_대분류_공산품_총구매금액,공휴일_대분류_남성의류_총구매금액,공휴일_대분류_남성정장스포츠_총구매금액,공휴일_대분류_로얄부틱_총구매금액,공휴일_대분류_명품잡화_총구매금액,공휴일_대분류_생식품_총구매금액,공휴일_대분류_스포츠캐쥬얼_총구매금액,공휴일_대분류_아동_총구매금액,공휴일_대분류_아동_스포츠_총구매금액,공휴일_대분류_아동문화_총구매금액,공휴일_대분류_여성의류파트_총구매금액,공휴일_대분류_여성정장_총구매금액,공휴일_대분류_여성캐쥬얼_총구매금액,공휴일_대분류_영라이브_총구매금액,공휴일_대분류_영어덜트캐쥬얼_총구매금액,공휴일_대분류_영캐릭터_총구매금액,공휴일_대분류_영플라자_총구매금액,공휴일_대분류_잡화_총구매금액,공휴일_대분류_케주얼_구두_아동_총구매금액,공휴일_대분류_패션잡화_총구매금액,공휴일_대분류_가정용품_구매횟수,공휴일_대분류_골프_유니캐쥬얼_구매횟수,공휴일_대분류_공산품_구매횟수,공휴일_대분류_남성의류_구매횟수,공휴일_대분류_남성정장스포츠_구매횟수,공휴일_대분류_로얄부틱_구매횟수,공휴일_대분류_명품잡화_구매횟수,공휴일_대분류_생식품_구매횟수,공휴일_대분류_스포츠캐쥬얼_구매횟수,공휴일_대분류_아동_구매횟수,공휴일_대분류_아동_스포츠_구매횟수,공휴일_대분류_아동문화_구매횟수,공휴일_대분류_여성의류파트_구매횟수,공휴일_대분류_여성정장_구매횟수,공휴일_대분류_여성캐쥬얼_구매횟수,공휴일_대분류_영라이브_구매횟수,공휴일_대분류_영어덜트캐쥬얼_구매횟수,공휴일_대분류_영캐릭터_구매횟수,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.482151,1.851040,-1.345509,-0.057297,0.017620,-0.597009,0.704620,-0.413647,0.071850,-0.597009,0.704620,-0.413647,0.071850,0.567827,0.092192,-0.203556,-0.313132,-0.519468,-0.491469,-0.236981,-0.576086,-0.622120,-0.670149,-0.282014,0.421006,0.050936,-0.596434,1.047253,-0.698995,1.204317,1.324717,-0.410712,-0.605861,-0.629268,-0.651028,-0.432991,-0.062820,-0.262171,-0.584917,...,-0.090195,-0.110797,-0.235776,-0.126524,-0.101251,-0.067885,-0.140549,-0.20149,-0.11668,-0.082473,-0.111084,-0.086907,-0.092340,-0.062198,-0.134168,-0.095225,-0.068361,-0.081019,-0.126840,-0.084245,-0.174285,-0.162678,-0.222962,-0.150631,-0.301801,-0.179448,-0.139182,-0.105096,-0.225863,-0.252797,-0.165962,-0.110783,-0.148917,-0.121695,-0.128685,-0.138524,-0.211595,-0.116973,-0.108769,-0.10576,-0.162079,-0.286379,-0.209907,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,0.109631,-0.482151,0.924134,1.132588,0.222706,1.176010,-0.231765,0.715412,-0.896636,0.333357,-0.231765,0.715412,-0.896636,0.333357,-0.429672,0.586200,-0.083161,0.523032,0.601863,0.544216,-0.256440,-0.576086,0.433398,-0.383294,0.579383,-0.500985,-0.521621,0.510469,1.370221,-0.328741,0.023169,-0.116958,-0.178962,-0.605861,0.747031,-0.291339,0.885621,-0.451041,-0.455061,0.637478,...,-0.090195,-0.110797,-0.235776,-0.126524,-0.101251,-0.067885,-0.140549,-0.20149,-0.11668,-0.082473,-0.111084,-0.086907,-0.092340,-0.062198,-0.134168,-0.095225,-0.068361,-0.081019,2.546882,-0.084245,-0.174285,3.590341,-0.222962,-0.150631,-0.301801,-0.179448,-0.139182,-0.105096,-0.225863,-0.252797,-0.165962,-0.110783,-0.148917,-0.121695,-0.128685,-0.138524,-0.211595,-0.116973,-0.108769,-0.10576,2.389509,-0.286379,-0.209907,4.161199,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.482151,-0.620708,-0.106461,0.277707,1.465608,-0.126036,-0.102531,0.545699,-0.371335,-0.126036,-0.102531,0.545699,-0.371335,-0.412172,0.562013,1.180992,2.752802,0.483828,2.680316,0.214679,0.184729,0.220442,0.439526,0.579383,-0.290401,-0.239110,-0.351750,0.020979,-0.357972,-0.149513,-0.481170,2.138532,2.022656,2.352713,3.125710,3.522845,0.519512,0.702281,0.148520,...,-0.090195,1.637603,-0.235776,-0.126524,-0.101251,-0.067885,20.978316,-0.20149,-0.11668,-0.082473,1.192858,-0.086907,-0.092340,-0.062198,-0.134168,-0.095225,-0.068361,-0.081019,-0.126840,-0.084245,-0.174285,-0.162678,-0.222962,2.490

In [147]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


### 피처셀렉션

In [52]:
# sp - 각 Feature와 Target 사이에 유의한 통계적 관계가 있는지 계산하여 Features를 선택
#from tqdm.notebook import tqdm
#params = {
#    'n_estimators': 891,
#    'learning_rate': 0.019329933105546135,
#    'num_leaves': 117,
#    'max_depth': 6,
#    'min_child_samples': 71,
#    'subsample': 0.8999999999999999,
#    'colsample_bytree': 0.8999999999999999
#    }
#model = LGBMClassifier(**params, random_state=42)
#cv = KFold(n_splits=5, shuffle=True, random_state=42)
#
#scores = []
#for percentile in tqdm(range(1,101)):
#    sp = SelectPercentile(percentile=percentile)
#    x = sp.fit_transform(train_ft, target)
#    score = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
#    scores.append([percentile, score])
#scores = np.array(scores)
#idx = np.argmax(scores[:,1])
#best_score = scores[idx]
#
#sp = SelectPercentile(percentile=best_score[0])
#sp.fit(train_ft,target)
#best_cols = sp.get_feature_names_out()
#best_cols

In [62]:
# selectfrommodel - 모델의 Features 중요도 또는 가중치를 기반으로 Features를 선택
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold, cross_val_score

cv = KFold(n_splits=5, shuffle=True, random_state=42)

params = {
   'n_estimators': 891,
   'learning_rate': 0.019329933105546135,
   'num_leaves': 117,
   'max_depth': 6,
   'min_child_samples': 71,
   'subsample': 0.8999999999999999,
   'colsample_bytree': 0.8999999999999999,
   #'early_stopping_rounds' : 30
   }
rf = RandomForestClassifier(random_state=42) # 특성 선택을 하기 위한 모델
model = LGBMClassifier(**params, random_state=42)

fs = SelectFromModel(rf) # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
x_train = fs.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환
scores = cross_val_score(model, x_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
print(scores.mean())

best_cols = fs.get_feature_names_out()

0.7176159478587987


In [63]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

cv = KFold(n_splits=5, shuffle=True, random_state=42)

params = {
    'n_estimators': 800,
    'learning_rate': 0.04036413044768581,
    'max_depth': 4,
    'min_child_weight': 7,
    'subsample': 0.7505214930635562,
    'colsample_bytree': 0.6290102054237857,
    'gamma': 0.648553153047272
    }

rf = RandomForestClassifier(random_state=42) # 특성 선택을 하기 위한 모델
model = XGBClassifier(**params, random_state=42)

fs = SelectFromModel(rf) # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
x_train = fs.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환
scores = cross_val_score(model, x_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
print(scores.mean())

best_cols = fs.get_feature_names_out()

0.7191294444544921


In [64]:
len(best_cols)

362

In [65]:
train_ft[best_cols]

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,주구매_월,주구매_시간대,일별평균구매횟수,거래개월수,9시_12시_구매비율,12시_15시_구매비율,15시_18시_구매비율,18시_21시_구매비율,9시_12시_구매횟수,12시_15시_구매횟수,15시_18시_구매횟수,18시_21시_구매횟수,월초_구매비율,월말_구매비율,월초_구매횟수,월말_구매횟수,웨딩성수기_구매비율,웨딩성수기_구매횟수,1월_구매비율,2월_구매비율,3월_구매비율,4월_구매비율,5월_구매비율,6월_구매비율,7월_구매비율,8월_구매비율,9월_구매비율,10월_구매비율,11월_구매비율,12월_구매비율,1월_구매횟수,2월_구매횟수,3월_구매횟수,4월_구매횟수,5월_구매횟수,6월_구매횟수,7월_구매횟수,8월_구매횟수,9월_구매횟수,...,pv_중분류_골프웨어_구입금액비중,pv_중분류_골프(국내)_구입금액비중,pv_중분류_진캐쥬얼_구입금액비중,pv_중분류_색조화장품_구입금액비중,pv_중분류_양말_구입금액비중,pv_중분류_영트랜드_구입금액비중,pv_중분류_핸드백_구입금액비중,pv_중분류_란제리_구입금액비중,pv_중분류_스타킹_구입금액비중,pv_중분류_수입향수_구입금액비중,pv_중분류_신생아_구입금액비중,pv_중분류_셔츠_구입금액비중,pv_중분류_영캐주얼_구입금액비중,pv_중분류_생선_구입금액비중,pv_중분류_스카프_구입금액비중,pv_중분류_하이캐쥬얼_구입금액비중,pv_중분류_트래디셔널_구입금액비중,pv_중분류_캐릭터_구입금액비중,pv_중분류_유아복_구입금액비중,pv_중분류_헤어ACC_구입금액비중,pv_중분류_청과_구입금액비중,pv_중분류_일반조리_구입금액비중,pv_중분류_캐릭터캐주얼_구입금액비중,pv_중분류_아동_란제리_구입금액비중,pv_중분류_수입ACC_구입금액비중,pv_중분류_스포츠웨어_구입금액비중,pv_중분류_캐쥬얼구두_구입금액비중,pv_중분류_여성구두_구입금액비중,pv_중분류_넥타이_구입금액비중,pv_중분류_욕실용품_구입금액비중,pv_중분류_수입의류_구입금액비중,pv_중분류_싸롱화_구입금액비중,pv_중분류_건강식품_구입금액비중,pv_중분류_스포츠캐주얼_구입금액비중,pv_중분류_토탈_구입금액비중,pv_중분류_스포츠슈즈_구입금액비중,pv_중분류_우산_장갑_구입금액비중,pv_중분류_영커리어캐주얼_구입금액비중,pv_중분류_내셔날_구입금액비중,중분류_란제리_구매주기,중분류_상품군미지정_구매주기,중분류_수입종합화장품_구매주기,중분류_아동_구매주기,중분류_야채_구매주기,중분류_영캐주얼_구매주기,중분류_용기보증_구매주기,중분류_진캐쥬얼_구매주기,공휴일_대분류_잡화_총구매금액,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,0.109631,1.851040,-1.345509,-0.057297,0.017620,-0.597009,0.704620,-0.413647,0.071850,-0.597009,0.704620,-0.413647,0.071850,0.567827,0.092192,-0.203556,-0.313132,-0.519468,-0.491469,-0.236981,-0.576086,-0.622120,-0.670149,-0.282014,0.421006,0.050936,-0.596434,1.047253,-0.698995,1.204317,1.324717,-0.410712,-0.605861,-0.629268,-0.651028,-0.432991,-0.062820,-0.262171,-0.584917,0.310375,...,3.421259,-0.194156,-0.395673,-0.142969,0.468549,-0.238324,-0.281923,-0.293551,-0.185710,-0.101131,-0.149364,-0.255356,-0.424275,-0.180258,-0.171397,-0.202312,-0.216139,-0.340170,-0.188143,-0.195271,-0.236442,-0.184130,-0.172952,-0.176855,-0.174793,-0.197176,-0.203008,-0.211031,-0.132177,-0.184968,-0.206930,0.405469,-0.159103,-0.168985,-0.150546,-0.181325,-0.120714,-0.164275,-0.206048,-0.273690,-0.311303,-0.760661,-0.230655,-0.286267,-0.394979,-0.511877,-0.371191,-0.084245,-0.741478,-1.315250
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.924134,1.132588,0.222706,1.176010,-0.231765,0.715412,-0.896636,0.333357,-0.231765,0.715412,-0.896636,0.333357,-0.429672,0.586200,-0.083161,0.523032,0.601863,0.544216,-0.256440,-0.576086,0.433398,-0.383294,0.579383,-0.500985,-0.521621,0.510469,1.370221,-0.328741,0.023169,-0.116958,-0.178962,-0.605861,0.747031,-0.291339,0.885621,-0.451041,-0.455061,0.637478,1.741547,...,0.484997,-0.194156,0.962763,-0.142969,-0.151849,-0.238324,-0.281923,0.582062,-0.185710,-0.101131,-0.149364,-0.255356,1.550202,-0.180258,-0.171397,-0.202312,0.236596,-0.104021,-0.188143,-0.195271,-0.098809,-0.184130,-0.172952,-0.176855,-0.174793,-0.197176,-0.203008,1.265321,-0.132177,-0.184968,-0.206930,0.526056,-0.159103,0.651864,2.799254,-0.181325,-0.120714,0.065357,-0.206048,8.788667,-0.311303,0.034829,-0.230655,-0.286267,1.346353,-0.511877,-0.246614,-0.084245,-0.448069,-0.626416
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.620708,-0.106461,0.277707,1.465608,-0.126036,-0.102531,0.545699,-0.371335,-0.126036,-0.102531,0.545699,-0.371335,-0.412172,0.562013,1.180992,2.752802,0.483828,2.680316,0.214679,0.184729,0.220442,0.439526,0.579383,-0.290401,-0.239110,-0.351750,0.020979,-0.357972,-0.149513,-0.481170,2.138532,2.022656,2.352713,3.125710,3.522845,0.519512,0.702281,0.148520,1.503018,...,-0.224183,2.053579,-0.074158,-0.142969,-0.054415,-0.238324,-0.137269,0.263839,-0.145221,-0.101131,-0.149364,-0.229176,-0.205059,-0.180258,-0.171397,-0.202312,0.674066,-0.057560,-0.188143,-0.195271,-0.236442,-0.078654,-0.172952,-0.176855,-0.1747

### CV점수

In [123]:
#피처 셀렉션 한거
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

params = {
    'n_estimators': 891,
    'learning_rate': 0.019329933105546135,
    'num_leaves': 117,
    'max_depth': 6,
    'min_child_samples': 71,
    'subsample': 0.8999999999999999,
    'colsample_bytree': 0.8999999999999999,
    'early_stopping_rounds' : 30
}

# 모델 훈련 및 특성 선택
rf = RandomForestClassifier(random_state=42)
fs = SelectFromModel(rf)
x_train_selected = fs.fit_transform(train_ft, target)

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(x_train_selected, target):
    # 학습 데이터와 검증 데이터 분리 (numpy 방식으로 인덱싱)
    x_train = x_train_selected[tri]
    y_train = target.iloc[tri]
    x_valid = x_train_selected[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = LGBMClassifier(**params)
    model.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],  # 검증 데이터 제공
        eval_metric="f1_macro"
    )

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.125705 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 68966
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 362
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392152 -> initscore=-0.438276
[LightGBM] [Info] Start training from score -0.438276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

###모델링

In [136]:
#피처셀렉션 한거
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score
import numpy as np



params = {
    'n_estimators': 800,
    'learning_rate': 0.04036413044768581,
    'max_depth': 4,
    'min_child_weight': 7,
    'subsample': 0.7505214930635562,
    'colsample_bytree': 0.6290102054237857,
    'gamma': 0.648553153047272
    }

# 모델 훈련 및 특성 선택
rf = RandomForestClassifier(random_state=42)
fs = SelectFromModel(rf)
x_train_selected = fs.fit_transform(train_ft, target)

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(x_train_selected, target):
    # 학습 데이터와 검증 데이터 분리 (numpy 방식으로 인덱싱)
    x_train = x_train_selected[tri]
    y_train = target.iloc[tri]
    x_valid = x_train_selected[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)
    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


Mean F1 Macro Score: 0.7184792477365533


In [150]:
# 피처셀렉션 안한거
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score
import numpy as np

params = {
    'n_estimators': 800,
    'learning_rate': 0.04036413044768581,
    'max_depth': 4,
    'min_child_weight': 7,
    'subsample': 0.7505214930635562,
    'colsample_bytree': 0.6290102054237857,
    'gamma': 0.648553153047272
    }


# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


Mean F1 Macro Score: 0.718389127465044


In [ ]:
#model = LGBMClassifier(**params)
#model.fit(train_ft, target)

In [ ]:
#pred = model.predict(test_ft)  # test_data는 테스트 데이터셋입니다.

In [ ]:
#submit["target"] = pred
#submit

In [ ]:
#submit.to_csv(f"{DATA_PATH}lgbm_1106_submit.csv",index=False)